In [ ]:
import sys

sys.path.append("..")

from core.DataLoader import DataPreprocessor, get_load_config_from_yaml
import numpy as np
from importlib import reload
import matplotlib.pyplot as plt
import yaml
import core.keras_models as Models
import core
import os


if False:
    PLOTS_DIR = "plots/ToponiumInfusedTransformer/"
    MODEL_DIR = "models/mixed_model"
elif True:
    PLOTS_DIR = "plots/RegressionTransformer_pil/"
else:
    PLOTS_DIR = "plots/RegressionTransformer_/"
    MODEL_DIR = "../models/regression_transformer_old/"


CONFIG_PATH = "../config/workspace_config.yaml"

# Create directories
os.makedirs(PLOTS_DIR, exist_ok=True)

# Set plotting style
plt.rcParams.update({"font.size": 20})
from core.utils import lorentz_vector_from_PtEtaPhiE_array
from core.evaluation import c_hel, c_han

# Load data
load_config = get_load_config_from_yaml(CONFIG_PATH)
DataProcessor = DataPreprocessor(load_config)
data_config = DataProcessor.load_from_npz(
    load_config.data_path["test"], event_numbers="odd", max_events=2_000_000
)
DataProcessor.add_custom_feature(
    (
        lambda X, data_config: c_hel(
            lorentz_vector_from_PtEtaPhiE_array(X["top_truth"][:, 0, :4]),
            lorentz_vector_from_PtEtaPhiE_array(X["top_truth"][:, 1, :4]),
            lorentz_vector_from_PtEtaPhiE_array(X["lepton_truth"][:, 0, :4]),
            lorentz_vector_from_PtEtaPhiE_array(X["lepton_truth"][:, 1, :4]),
        )
    ),
    "c_hel",
)
DataProcessor.add_custom_feature(
    (
        lambda X, data_config: c_han(
            lorentz_vector_from_PtEtaPhiE_array(X["top_truth"][:, 0, :4]),
            lorentz_vector_from_PtEtaPhiE_array(X["top_truth"][:, 1, :4]),
            lorentz_vector_from_PtEtaPhiE_array(X["lepton_truth"][:, 0, :4]),
            lorentz_vector_from_PtEtaPhiE_array(X["lepton_truth"][:, 1, :4]),
        )
    ),
    "c_han",
)

X, y = DataProcessor.get_data()
del DataProcessor

In [ ]:
import core.reconstruction.BaselineMethods as BaselineMethods
import core.evaluation.reconstruction_evaluator as Evaluation
from core.reconstruction import (
    GroundTruthReconstructor,
    PerfectAssignmentReconstructor,
    CompositeNeutrinoComponentReconstructor,
)

reload(Evaluation)
reload(BaselineMethods)
ground_truth_assigner = GroundTruthReconstructor(data_config, use_nu_flows=True, assignment_name="")

transformer = PerfectAssignmentReconstructor(
    data_config, neutrino_reco_name=r"Transformer (MagDir) $\nu$", assignment_name=""
)
MODEL_DIR = "../models/regression_transformer/"
transformer.load_model(f"{MODEL_DIR}/odd_model.keras")

transformer_cartesian = PerfectAssignmentReconstructor(
    data_config, neutrino_reco_name=r"Transformer (Cartesian) $\nu$", assignment_name=""
)
MODEL_DIR = "../models/regression_transformer_cartesian/"
transformer_cartesian.load_model(f"{MODEL_DIR}/odd_model.keras")

transformer_PtEtaPhi = PerfectAssignmentReconstructor(
    data_config, neutrino_reco_name=r"Transformer (PtEtaPhi) $\nu$", assignment_name=""
)
MODEL_DIR = "../models/regression_transformer_PtEtaPhi/"
transformer_PtEtaPhi.load_model(f"{MODEL_DIR}/odd_model.keras")


prediction_manager = Evaluation.PredictionManager(
    [ground_truth_assigner, transformer,transformer_cartesian,transformer_PtEtaPhi],
    X,
    y,
)

In [ ]:
from core.evaluation import reco_variable_config
reload(reco_variable_config)

evaluator = Evaluation.ReconstructionPlotter(prediction_manager, reco_variable_config.reconstruction_variable_configs)

In [ ]:
PLOTS_DIR = "plots/RegressionTransformer_CompareLosses/"
os.makedirs(PLOTS_DIR, exist_ok=True)

In [ ]:
#plt.rcParams.update({"font.size": 14})

fig, ax = evaluator.plot_variable_deviation("W_mass")
fig.savefig(os.path.join(PLOTS_DIR, "W_mass_deviation.pdf"), bbox_inches="tight")
fig, ax = evaluator.plot_variable_distribution("W_mass", xlims=(40, 120))
fig.savefig(os.path.join(PLOTS_DIR, "W_mass_distribution.pdf"), bbox_inches="tight")

In [ ]:
fig, ax = evaluator.plot_variable_deviation("W_energy")
fig.savefig(os.path.join(PLOTS_DIR, "W_energy_deviation.pdf"), bbox_inches="tight")
fig, ax = evaluator.plot_variable_distribution("W_energy")
fig.savefig(os.path.join(PLOTS_DIR, "W_energy_distribution.pdf"), bbox_inches="tight")

In [ ]:
fig, ax = evaluator.plot_binned_variable(
    "c_hel",
    "deviation",
    "non_training",
    "truth_ttbar_mass",
    xlims=(340e3, 800e3),
    bins=10,
    fancy_feature_label=r"$m(t\bar{t})$ [GeV]",
)
ticks = ax.get_xticks()
ax.set_xticks(ticks)
ax.set_xticklabels([f"{int(tick/1e3)}" for tick in ticks])
ax.set_xlim(340e3, 800e3)
fig.savefig(f"{PLOTS_DIR}/c_hel_deviation_vs_ttbar_mass.pdf", bbox_inches="tight")
fig, ax = evaluator.plot_binned_variable(
    "c_han",
    "deviation",
    "non_training",
    "truth_ttbar_mass",
    xlims=(340e3, 800e3),
    bins=10,
    fancy_feature_label=r"$m(t\bar{t})$ [GeV]",
)
ticks = ax.get_xticks()
ax.set_xticks(ticks)
ax.set_xticklabels([f"{int(tick/1e3)}" for tick in ticks])
ax.set_xlim(340e3, 800e3)
fig.savefig(f"{PLOTS_DIR}/c_han_deviation_vs_ttbar_mass.pdf", bbox_inches="tight")
fig, ax = evaluator.plot_binned_variable(
    "c_hel",
    "resolution",
    "non_training",
    "truth_ttbar_mass",
    xlims=(340e3, 800e3),
    bins=10,
    fancy_feature_label=r"$m(t\bar{t})$ [GeV]",
)
ticks = ax.get_xticks()
ax.set_xticks(ticks)
ax.set_xticklabels([f"{int(tick/1e3)}" for tick in ticks])
ax.set_xlim(340e3, 800e3)
fig.savefig(f"{PLOTS_DIR}/c_hel_resolution_vs_ttbar_mass.pdf", bbox_inches="tight")
fig, ax = evaluator.plot_binned_variable(
    "c_han",
    "resolution",
    "non_training",
    "truth_ttbar_mass",
    xlims=(340e3, 800e3),
    bins=10,
    fancy_feature_label=r"$m(t\bar{t})$ [GeV]",
)
ticks = ax.get_xticks()
ax.set_xticks(ticks)
ax.set_xticklabels([f"{int(tick/1e3)}" for tick in ticks])
ax.set_xlim(340e3, 800e3)
fig.savefig(f"{PLOTS_DIR}/c_han_resolution_vs_ttbar_mass.pdf", bbox_inches="tight")

In [ ]:
fig, ax = evaluator.plot_binned_variable(
    "c_hel",
    "deviation",
    "custom",
    "c_hel",
    xlims=(-1, 1),
    bins=10,
    fancy_feature_label=r"$c_{hel}$",
)
fig.savefig(f"{PLOTS_DIR}/c_hel_deviation_vs_c_hel.pdf", bbox_inches="tight")
fig, ax = evaluator.plot_binned_variable(
    "c_han",
    "deviation",
    "custom",
    "c_han",
    xlims=(-1, 1),
    bins=10,
    fancy_feature_label=r"$c_{han}$",
)
ticks = ax.get_xticks()
fig.savefig(f"{PLOTS_DIR}/c_han_deviation_vs_c_han.pdf", bbox_inches="tight")

In [ ]:
fig, ax = evaluator.plot_binned_variable(
    "c_hel",
    "resolution",
    "custom",
    "c_hel",
    xlims=(-1, 1),
    bins=10,
    fancy_feature_label=r"$c_{hel}$",
)
fig.savefig(f"{PLOTS_DIR}/c_hel_resolution_vs_c_hel.pdf", bbox_inches="tight")
fig, ax = evaluator.plot_binned_variable(
    "c_han",
    "resolution",
    "custom",
    "c_han",
    xlims=(-1, 1),
    bins=10,
    fancy_feature_label=r"$c_{han}$",
)
ticks = ax.get_xticks()
fig.savefig(f"{PLOTS_DIR}/c_han_resolution_vs_c_han.pdf", bbox_inches="tight")

In [ ]:
fig, ax = evaluator.plot_variable_deviation("top_energy")
fig.savefig(f"{PLOTS_DIR}/top_energy_deviation.pdf", bbox_inches="tight")

fig, ax = evaluator.plot_variable_deviation("top_mass")
fig.savefig(f"{PLOTS_DIR}/top_mass_deviation.pdf", bbox_inches="tight")

fig, ax = evaluator.plot_variable_deviation("top_gamma")
fig.savefig(f"{PLOTS_DIR}/top_gamma_deviation.pdf", bbox_inches="tight")

In [ ]:
fig, ax = evaluator.plot_variable_confusion_matrix(
    "c_hel", normalize="all", figsize_per_plot=(5.4, 5), bins=20, plot_mean=True
)
fig.savefig(os.path.join(PLOTS_DIR, "c_hel_confusion_matrix.pdf"))

fig, ax = evaluator.plot_variable_confusion_matrix(
    "c_han", normalize="all", figsize_per_plot=(5.4, 5), bins=20, plot_mean=True
)
fig.savefig(os.path.join(PLOTS_DIR, "c_han_confusion_matrix.pdf"))

In [ ]:
fig, ax = evaluator.plot_variable_distribution("c_hel", bins=20)
fig.savefig(f"{PLOTS_DIR}/c_hel_distribution.pdf", bbox_inches="tight")

fig, ax = evaluator.plot_variable_distribution("c_han", bins=20)
fig.savefig(f"{PLOTS_DIR}/c_han_distribution.pdf", bbox_inches="tight")

In [ ]:
fig, ax = evaluator.plot_variable_deviation("c_hel", xlims=(-1, 1))
fig.savefig(f"{PLOTS_DIR}/c_hel_deviation.pdf", bbox_inches="tight")

fig, ax = evaluator.plot_variable_deviation("c_han", xlims=(-1, 1))
fig.savefig(f"{PLOTS_DIR}/c_han_deviation.pdf", bbox_inches="tight")

In [ ]:
fig, ax = evaluator.plot_variable_deviation("ttbar_mass", xlims=(-300,300), use_relative_deviation=False)
fig.savefig(f"{PLOTS_DIR}/ttbar_mass_deviation.pdf", bbox_inches="tight")

fig, ax = evaluator.plot_variable_distribution("ttbar_mass", xlims = (0,1200))
fig.savefig(os.path.join(PLOTS_DIR, "ttbar_mass_distribution.pdf"), bbox_inches="tight")

In [ ]:
fig, ax = evaluator.plot_variable_deviation("cos_angle_nu_lep", xlims=(-1, 1))
fig.savefig(f"{PLOTS_DIR}/cos_angle_nu_lep_deviation.pdf", bbox_inches="tight")

fig, ax = evaluator.plot_variable_distribution("cos_angle_nu_lep", xlims = (-1,1))
fig.savefig(os.path.join(PLOTS_DIR, "cos_angle_nu_lep_distribution.pdf"), bbox_inches="tight")

In [ ]:
fig , ax = evaluator.plot_variable_deviation("nu_mag", xlims=(-100, 100), use_relative_deviation=False)
fig.savefig(f"{PLOTS_DIR}/nu_mag_deviation.pdf", bbox_inches="tight")

fig, ax = evaluator.plot_variable_distribution("nu_mag", xlims = (0,400))
fig.savefig(os.path.join(PLOTS_DIR, "nu_mag_distribution.pdf"), bbox_inches="tight")

In [ ]:
fig, ax = evaluator.plot_variable_deviation("cos_angle_nunubar_bbarll", xlims=(-1, 1))
fig.savefig(f"{PLOTS_DIR}/cos_angle_nunubar_bbarll_deviation.pdf", bbox_inches="tight")

fig, ax = evaluator.plot_variable_distribution("cos_angle_nunubar_bbarll", xlims=(-1, 1))
fig.savefig(os.path.join(PLOTS_DIR, "cos_angle_nunubar_bbarll_distribution.pdf"), bbox_inches="tight")

In [ ]:
fig, ax = evaluator.plot_variable_deviation("nunubar_mag", xlims=(-100, 100), use_relative_deviation=False)
fig.savefig(f"{PLOTS_DIR}/nunubar_mag_deviation.pdf", bbox_inches="tight")

fig, ax = evaluator.plot_variable_distribution("nunubar_mag", xlims=(0, 400))
fig.savefig(os.path.join(PLOTS_DIR, "nunubar_mag_distribution.pdf"), bbox_inches="tight")